In [1]:
from spacy.lang.en import English
import spacy
spacy.prefer_gpu()
from collections import Counter
import csv
import config
import tensorflow as tf
import numpy as np
import pandas as pd
import random as rnd
import subprocess
import os
import nltk
from nltk.corpus import stopwords
import re
import string
from collections import OrderedDict, defaultdict
from sklearn.linear_model import LogisticRegression
import fnmatch
#from tensorflow.keras.preprocessing.text import Tokenizer
from src.CustomTokenizer import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import json
from tqdm import tqdm
from data_processing.preprocessing import get_dataset_text, get_word_index_list, write_index_to_file, read_index_as_string

In [2]:

'''
vocab_base_folder_list = ['/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_0',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_200',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_400',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_600',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_800',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1000',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1200',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1400',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1600',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1800',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_2000']

training_files = ['/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_0.csv',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_200.csv',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_400.csv',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_600.csv',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_800.csv',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_1000.csv',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_1200.csv',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_1400.csv',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_1600.csv',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_1800.csv',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_2000.csv']

results =  ['/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/MPL_0',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/MPL_200',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/MPL_400',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/MPL_600',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/MPL_800',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/MPL_1000',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/MPL_1200',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/MPL_1400',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/MPL_1600',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/MPL_1800',
'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/MPL_2000']
for training_file_path,vocab_base_folder in zip(training_files, vocab_base_folder_list):
    print(training_file_path)
    vocab_path_list = [vocab_base_folder+"/"+opt for opt in ['opt1', 'opt2', 'opt3', 'opt4']]'''

'\nvocab_base_folder_list = [\'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_0\',\n\'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_200\',\n\'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_400\',\n\'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_600\',\n\'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_800\',\n\'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1000\',\n\'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1200\',\n\'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1400\',\n\'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1600\',\n\'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1800\',\n\'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_2000\']\n\ntraining_files = [\'/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training

## Load Training set

In [3]:
MPL_list = [str(i) for i in range(0,2200,200)]
vocab_base = '/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/'
training_base = '/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/'
results_base = '/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/'

vocab_base_folder_list = [vocab_base + "MPL_" + i for i in MPL_list]
training_files = [training_base+ "training_" + i + ".csv" for i in MPL_list]
results_folder_list = [results_base + "MPL_" + i for i in MPL_list]

In [4]:
for i in vocab_base_folder_list:
    print(i)
for i in training_files:
    print(i)
for i in results_folder_list:
    print(i)

/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_0
/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_200
/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_400
/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_600
/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_800
/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1000
/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1200
/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1400
/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1600
/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_1800
/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_2000
/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_0.csv
/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_200.csv
/home/sukanya/PhD/Results

In [22]:
def evaluate(training_file_path, testing_file_path, vocab_folder, results_folder):
    print("Loading training file from path: ", training_file_path)
    training_data = pd.read_csv(training_file_path)
    print(training_data.shape)
    print("Loading testing file from path: ", config.config_io.get('pan_19_processed_test'))
    testing_data = pd.read_csv(testing_file_path)
    print(testing_data.shape)
    feature_set_files = get_feature_set_files(vocab_folder, vocab_size = range(50, 550,50))
    metrics = {}
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    for key in list(feature_set_files.keys()):
        print(key)
        temp_metrics_dict={}
        vocab_files = []
        training_acc = []
        training_loss = []
        validation_loss = []
        validation_acc = []
        testing_acc= []
        testing_loss = []
        vocab_size_list = []
        #print(key, feature_set_files.get(key))
        for feature_file in feature_set_files.get(key):
            print("Evaluating:", feature_file)
            word_tokenizer = Tokenizer(analyzer=custom_analyzer)
            with open(feature_file, "r") as f:
                word_index_str = f.read().replace('\n', '')
            print("WORD INDEX STR:",word_index_str)
            word_tokenizer.fit_on_texts([word_index_str])
            len_train = len(training_data)
            # define the generators
            from src.DataGenerator import DataGenerator
            training_generator = DataGenerator(training_data.iloc[0:int(0.8*len_train)], tokenizer=word_tokenizer, batch_size=16)
            validation_generator = DataGenerator(training_data.iloc[int(0.8*len_train):], tokenizer=word_tokenizer, batch_size=16)
            testing_generator = DataGenerator(testing_data, tokenizer=word_tokenizer, batch_size=16)

            # this is a hack for "'DataGenerator' object has no attribute 'index'". It turns out that on_epoch_end creates the index that is used
            training_generator.on_epoch_end()
            validation_generator.on_epoch_end()
            testing_generator.on_epoch_end()
            # parameters
            num_classes =2
            num_features = 1

            # define model
            model = Sequential()
            model.add(Dense(50, input_dim=num_features, activation='relu', kernel_initializer='he_uniform'))
            model.add(Dense(num_classes, activation='softmax'))

            # compile model
            opt = SGD(lr=0.01, momentum=0.9)
            #model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

            # fit model
            history = model.fit(training_generator, validation_data=validation_generator, verbose=1, batch_size=16, 
                                 epochs=30, steps_per_epoch=100, callbacks=[callback]) #validation_steps=100,

            history_dict = history.history
            vocab_files.append(feature_file.split('/')[-1])
            json.dump(history_dict, open(results_folder+"/history_" + key + "_" +feature_file.split('/')[-1], 'w'))
            loss = history_dict['loss'][-1]
            training_loss.append(loss)
            acc = history_dict['accuracy'][-1]
            training_acc.append(acc)
            val_loss = history_dict['val_loss'][-1]
            validation_loss.append(val_loss)
            val_acc = history_dict['val_accuracy'][-1]
            validation_acc.append(val_acc)
            test_loss, test_acc = model.evaluate(testing_generator)
            testing_acc.append(test_acc)
            testing_loss.append(test_loss)
            vocab_size_list.append(feature_file.split('/')[-1])
            print("training acc: ", acc, ", training loss: ", loss, ", val acc: ", val_acc, ", val loss: ", val_loss,", test acc: ", test_acc, ", test loss: ", test_loss)
            print()
        temp_metrics_dict={'vocab_size':vocab_size_list,
                           'training_acc': training_acc,
                           'validation_acc': validation_acc,
                           'testing_acc':testing_acc,
                           'training_loss' : training_loss,
                           'validation_loss': validation_loss,
                           'testing_loss':testing_loss}
        json.dump(temp_metrics_dict, open(results_folder+"/total_history_" + key, 'w'))
        metrics[key] = temp_metrics_dict
    json.dump(metrics, open(results_folder+"/total_history_combined", 'w'))

In [13]:
for i,j,k in zip([1,2,3],[4,5,6],[7,8,9]):
    print(i,j,k)

1 4 7
2 5 8
3 6 9


In [14]:
testing_file_path = '/home/sukanya/PhD/Datasets/PAN SCD/pan19-style-change-detection/processed/test.csv'

In [24]:
for training_file_path,vocab_folder,results_folder in zip(training_files,vocab_base_folder_list,results_folder_list):
    print("Started evaluation for:", training_file_path)
    evaluate(training_file_path, testing_file_path, vocab_folder, results_folder)

Started evaluation for: /home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_0.csv
Loading training file from path:  /home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/datasets/training_0.csv
(18961, 6)
Loading testing file from path:  /home/sukanya/PhD/Datasets/PAN SCD/pan19-style-change-detection/processed/test.csv
(9538, 5)
opt1
Evaluating: /home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_0/opt1/50word_list.txt
WORD INDEX STR: the to a  of and is i in you that it $ for   this be with as on are have if not can or but your 's n't would an do so from there will at by they   one my which what all more we some when
Epoch 1/30
100/100 [==============================] - 8s 80ms/step - loss: 0.7042 - accuracy: 0.5581 - val_loss: 0.6925 - val_accuracy: 0.5680
Epoch 2/30
100/100 [==============================] - 8s 82ms/step - loss: 0.6968 - accuracy: 0.5050 - val_loss: 0.6903 - val_accuracy: 0.5614
Epoch 3/30
100/100 [=====================

100/100 [==============================] - 8s 78ms/step - loss: 0.6798 - accuracy: 0.5781 - val_loss: 0.6730 - val_accuracy: 0.5783
Epoch 13/30
100/100 [==============================] - 8s 81ms/step - loss: 0.6724 - accuracy: 0.5794 - val_loss: 0.6723 - val_accuracy: 0.5791
Epoch 14/30
100/100 [==============================] - 8s 80ms/step - loss: 0.6865 - accuracy: 0.5494 - val_loss: 0.6742 - val_accuracy: 0.5794
Epoch 15/30
100/100 [==============================] - 8s 81ms/step - loss: 0.6852 - accuracy: 0.5425 - val_loss: 0.6717 - val_accuracy: 0.5725
Epoch 16/30
100/100 [==============================] - 8s 78ms/step - loss: 0.6762 - accuracy: 0.5656 - val_loss: 0.6712 - val_accuracy: 0.5614
Epoch 17/30
100/100 [==============================] - 8s 78ms/step - loss: 0.6600 - accuracy: 0.6187 - val_loss: 0.6729 - val_accuracy: 0.5794
Epoch 18/30
100/100 [==============================] - 8s 81ms/step - loss: 0.6716 - accuracy: 0.5969 - val_loss: 0.6708 - val_accuracy: 0.5725
Epoc

Epoch 28/30
100/100 [==============================] - 8s 82ms/step - loss: 0.6671 - accuracy: 0.5750 - val_loss: 0.6690 - val_accuracy: 0.5625
Epoch 29/30
100/100 [==============================] - 8s 78ms/step - loss: 0.6839 - accuracy: 0.5500 - val_loss: 0.6679 - val_accuracy: 0.5804
Epoch 30/30
596/596 [==============================] - 14s 24ms/step - loss: 0.6696 - accuracy: 0.5924
training acc:  0.5787500143051147 , training loss:  0.6685338020324707 , val acc:  0.5791139006614685 , val loss:  0.6713536977767944 , test acc:  0.5923867225646973 , test loss:  0.669553816318512

Evaluating: /home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_0/opt1/200word_list.txt
WORD INDEX STR: the to a  of and is i in you that it $ for   this be with as on are have if not can or but your 's n't would an do so from there will at by they   one my which what all more we some when then like was use / has just any also about get no how only could other time out does them than up nee

Epoch 8/30
100/100 [==============================] - 8s 80ms/step - loss: 0.6711 - accuracy: 0.5944 - val_loss: 0.6722 - val_accuracy: 0.5678
Epoch 9/30
100/100 [==============================] - 8s 80ms/step - loss: 0.6629 - accuracy: 0.5925 - val_loss: 0.6876 - val_accuracy: 0.5794
Epoch 10/30
100/100 [==============================] - 8s 79ms/step - loss: 0.6787 - accuracy: 0.5744 - val_loss: 0.6710 - val_accuracy: 0.5696
Epoch 11/30
100/100 [==============================] - 8s 81ms/step - loss: 0.6818 - accuracy: 0.5425 - val_loss: 0.6705 - val_accuracy: 0.5630
Epoch 12/30
100/100 [==============================] - 8s 80ms/step - loss: 0.6696 - accuracy: 0.6056 - val_loss: 0.6702 - val_accuracy: 0.5643
Epoch 13/30
100/100 [==============================] - 8s 79ms/step - loss: 0.6768 - accuracy: 0.5738 - val_loss: 0.6713 - val_accuracy: 0.5902
Epoch 14/30
100/100 [==============================] - 8s 82ms/step - loss: 0.6647 - accuracy: 0.5931 - val_loss: 0.6712 - val_accuracy: 0

Epoch 1/30
100/100 [==============================] - 8s 82ms/step - loss: 0.6936 - accuracy: 0.5437 - val_loss: 0.6858 - val_accuracy: 0.5870
Epoch 2/30
100/100 [==============================] - 8s 81ms/step - loss: 0.6874 - accuracy: 0.5688 - val_loss: 0.6812 - val_accuracy: 0.5630
Epoch 3/30
100/100 [==============================] - 8s 81ms/step - loss: 0.6824 - accuracy: 0.5806 - val_loss: 0.6822 - val_accuracy: 0.5860
Epoch 4/30
100/100 [==============================] - 8s 83ms/step - loss: 0.6771 - accuracy: 0.5850 - val_loss: 0.6753 - val_accuracy: 0.5701
Epoch 5/30
100/100 [==============================] - 8s 81ms/step - loss: 0.6693 - accuracy: 0.6012 - val_loss: 0.6803 - val_accuracy: 0.5794
Epoch 6/30
100/100 [==============================] - 8s 82ms/step - loss: 0.6841 - accuracy: 0.5706 - val_loss: 0.6756 - val_accuracy: 0.5794
Epoch 7/30
100/100 [==============================] - 8s 81ms/step - loss: 0.6608 - accuracy: 0.6306 - val_loss: 0.6730 - val_accuracy: 0.5794

...

Epoch 1/30
596/596 [==============================] - 14s 24ms/step - loss: 0.6997 - accuracy: 0.5024
training acc:  0.3405172526836395 , training loss:  0.9165400266647339 , val acc:  0.2678571343421936 , val loss:  0.760252833366394 , test acc:  0.5024119019508362 , test loss:  0.6996743083000183

Evaluating: /home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_2000/opt3/400word_list.txt
WORD INDEX STR:  $   would 's n't one use like also / time could need get - even might make way two people want work may using first much think   something see different really = good know used 'm example 1 well probably set many case since point question things however try 're still enough take say number 've 2 system better find actually another possible without answer new power look data go code long let problem right 'd back \mathcal able value image going high either less 3 part lens sure form ca change best means + & thing always 'll around space give page given small create lik

596/596 [==============================] - 15s 25ms/step - loss: 0.8062 - accuracy: 0.4915
training acc:  0.3556034564971924 , training loss:  1.5159751176834106 , val acc:  0.375 , val loss:  1.219765543937683 , test acc:  0.49150586128234863 , test loss:  0.8061934113502502

Evaluating: /home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_2000/opt3/500word_list.txt
WORD INDEX STR:  $   would 's n't one use like also / time could need get - even might make way two people want work may using first much think   something see different really = good know used 'm example 1 well probably set many case since point question things however try 're still enough take say number 've 2 system better find actually another possible without answer new power look data go code long let problem right 'd back \mathcal able value image going high either less 3 part lens sure form ca change best means + & thing always 'll around space give page given small create likely mm large process of

596/596 [==============================] - 14s 24ms/step - loss: 0.6801 - accuracy: 0.5877
training acc:  0.40086206793785095 , training loss:  0.9331123232841492 , val acc:  0.6071428656578064 , val loss:  0.6714800596237183 , test acc:  0.5876677632331848 , test loss:  0.6801491975784302

Evaluating: /home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_2000/opt4/150word_list.txt
WORD INDEX STR: . ,  $   would 's n't one use like also / time could need get - even might make way two people want work may using first much think   something see different really = good know used 'm example 1 well probably set many case since point question things however try 're still enough take say number 've 2 system better find actually another possible without answer new power look data go code long let problem right 'd back \mathcal able value image going high either less 3 part lens sure form ca change best means + & thing always 'll around space give page given small create likely m

Epoch 1/30
596/596 [==============================] - 15s 24ms/step - loss: 0.6949 - accuracy: 0.5559
training acc:  0.642241358757019 , training loss:  0.6352719068527222 , val acc:  0.625 , val loss:  0.6342214941978455 , test acc:  0.5558934807777405 , test loss:  0.6949007511138916

Evaluating: /home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_2000/opt4/350word_list.txt
WORD INDEX STR: . ,  $   would 's n't one use like also / time could need get - even might make way two people want work may using first much think   something see different really = good know used 'm example 1 well probably set many case since point question things however try 're still enough take say number 've 2 system better find actually another possible without answer new power look data go code long let problem right 'd back \mathcal able value image going high either less 3 part lens sure form ca change best means + & thing always 'll around space give page given small create likely mm la

Epoch 1/30
596/596 [==============================] - 14s 24ms/step - loss: 0.6860 - accuracy: 0.5877
training acc:  0.48275861144065857 , training loss:  0.7417845726013184 , val acc:  0.6071428656578064 , val loss:  0.65009605884552 , test acc:  0.5876677632331848 , test loss:  0.685966432094574

Evaluating: /home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/vocab/MPL_2000/opt4/500word_list.txt
WORD INDEX STR: . ,  $   would 's n't one use like also / time could need get - even might make way two people want work may using first much think   something see different really = good know used 'm example 1 well probably set many case since point question things however try 're still enough take say number 've 2 system better find actually another possible without answer new power look data go code long let problem right 'd back \mathcal able value image going high either less 3 part lens sure form ca change best means + & thing always 'll around space give page given small create 

In [ ]:
'''print("Loading training file from path: ", config.config_io.get('pan_19_processed_train'))
training_data = pd.read_csv(config.config_io.get('pan_19_processed_train'))
training_data.shape'''

In [ ]:
'''training_data.head()'''

In [ ]:
'''training_data.columns'''

## Load Testing set

In [ ]:
'''print("Loading testing file from path: ", config.config_io.get('pan_19_processed_test'))
testing_data = pd.read_csv(config.config_io.get('pan_19_processed_test'))
testing_data.shape'''

In [ ]:
'''testing_data.head()'''

## Load the feature set files
We are doing it manually now for greater control. In the future, we must automate to read the files directly form the directory.

In [9]:
def get_feature_set_files (base_vocab_folder, vocab_size = range(50, 550,50)):
    opt1 = [base_vocab_folder+"/opt1/"+str(i)+"word_list.txt" for i in vocab_size]
    opt2 = [base_vocab_folder+"/opt2/"+str(i)+"word_list.txt" for i in vocab_size]
    opt3 = [base_vocab_folder+"/opt3/"+str(i)+"word_list.txt" for i in vocab_size]
    opt4 = [base_vocab_folder+"/opt4/"+str(i)+"word_list.txt" for i in vocab_size]
    return {'opt1': opt1, 'opt2': opt2, 'opt3': opt3, 'opt4': opt4}

In [ ]:
# index files
'''
feature_set_files = { 'opt1':
['/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt1/50word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt1/100word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt1/150word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt1/200word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt1/250word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt1/300word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt1/350word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt1/400word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt1/450word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt1/500word_list.txt'],
                    'opt2':
['/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt2/50word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt2/100word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt2/150word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt2/200word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt2/250word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt2/300word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt2/350word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt2/400word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt2/450word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt2/500word_list.txt'],
                    'opt3':
['/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt3/50word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt3/100word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt3/150word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt3/200word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt3/250word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt3/300word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt3/350word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt3/400word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt3/450word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt3/500word_list.txt'],
                    'opt4':
['/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt4/50word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt4/100word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt4/150word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt4/200word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt4/250word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt4/300word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt4/350word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt4/400word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt4/450word_list.txt',
'/home/sukanya/PycharmProjects/SiameseNNPAN/src/word_index/opt4/500word_list.txt']}'''


In [ ]:
'''nlp.pipeline'''

In [10]:
#nlp = English()
nlp = spacy.load("en_core_web_sm")
# takes micro seconds 90.7 µs ± 962 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each), 100 times faster than below loop
def custom_analyzer(text):
    #doc = nlp(text.lower())
    doc = list(nlp.pipe([text.lower()], disable=["tagger", "parser","ner"]))[0]
    return [t.text for t in doc]

print(custom_analyzer("A very large line this is I don't know how this works yet! God Save me!!"))

['a', 'very', 'large', 'line', 'this', 'is', 'i', 'do', "n't", 'know', 'how', 'this', 'works', 'yet', '!', 'god', 'save', 'me', '!', '!']


In [ ]:
# takes mili seconds 4.23 ms ± 124 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
'''def custom_analyzer(text):
    doc = nlp(text.lower())
    #doc = list(nlp.pipe([text.lower()], disable=["tagger", "parser","ner"]))[0]
    return [t.text for t in doc]

print(custom_analyzer("A very large line this is I don't know how this works yet! God Save me!!"))'''

In [ ]:
'''feature_set_files'''

In [ ]:
'''metrics = {}
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

for key in list(feature_set_files.keys()):
    print(key)
    temp_metrics_dict={}
    vocab_files = []
    training_acc = []
    training_loss = []
    validation_loss = []
    validation_acc = []
    testing_acc= []
    testing_loss = []
    vocab_size_list = []
    print(key, feature_set_files.get(key))
    for feature_file in feature_set_files.get(key):
        word_tokenizer = Tokenizer(analyzer=custom_analyzer)
        with open(feature_file, "r") as f:
            word_index_str = f.read().replace('\n', '')
        print(word_index_str)
        word_tokenizer.fit_on_texts([word_index_str])
        len_train = len(training_data)
        # define the generators
        from src.DataGenerator import DataGenerator
        training_generator = DataGenerator(training_data.iloc[0:int(0.8*len_train)], tokenizer=word_tokenizer, batch_size=16)
        validation_generator = DataGenerator(training_data.iloc[int(0.8*len_train):], tokenizer=word_tokenizer, batch_size=16)
        testing_generator = DataGenerator(testing_data, tokenizer=word_tokenizer, batch_size=16)

        # this is a hack for "'DataGenerator' object has no attribute 'index'". It turns out that on_epoch_end creates the index that is used
        training_generator.on_epoch_end()
        validation_generator.on_epoch_end()
        testing_generator.on_epoch_end()
        # parameters
        num_classes =2
        num_features = 1

        # define model
        model = Sequential()
        model.add(Dense(50, input_dim=num_features, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(num_classes, activation='softmax'))

        # compile model
        opt = SGD(lr=0.01, momentum=0.9)
        #model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

        # fit model
        history = model.fit(training_generator, validation_data=validation_generator, verbose=1, batch_size=16, 
                             epochs=30, steps_per_epoch=100, callbacks=[callback]) #validation_steps=100,

        history_dict = history.history
        vocab_files.append(feature_file.split('/')[-1])
        json.dump(history_dict, open("history_" + key + "_" +feature_file.split('/')[-1], 'w'))
        loss = history_dict['loss'][-1]
        training_loss.append(loss)
        acc = history_dict['accuracy'][-1]
        training_acc.append(acc)
        val_loss = history_dict['val_loss'][-1]
        validation_loss.append(val_loss)
        val_acc = history_dict['val_accuracy'][-1]
        validation_acc.append(val_acc)
        test_loss, test_acc = model.evaluate(testing_generator)
        testing_acc.append(test_acc)
        testing_loss.append(test_loss)
        vocab_size_list.append(feature_file.split('/')[-1])
        print("training acc: ", acc, ", training loss: ", loss, ", val acc: ", val_acc, ", val loss: ", val_loss,", test acc: ", test_acc, ", test loss: ", test_loss)
        print()
    temp_metrics_dict={'vocab_size':vocab_size_list,
        'training_loss' : training_loss,
         'training_acc': training_acc,
         'validation_loss': validation_loss,
         'validation_acc': validation_acc,
         'testing_loss':testing_loss,
         'testing_acc':testing_acc}
    json.dump(temp_metrics_dict, open("total_history_" + key, 'w'))
    metrics[key] = temp_metrics_dict
    '''

In [ ]:
'''json.dump(metrics, open("history/total_history_combined", 'w'))'''

In [ ]:
'''df_opt1 = pd.DataFrame(metrics.get('opt1'))
df_opt2 = pd.DataFrame(metrics.get('opt2'))
df_opt3 = pd.DataFrame(metrics.get('opt3'))
df_opt4 = pd.DataFrame(metrics.get('opt4'))'''

In [ ]:
'''df_opt1'''

In [ ]:
'''temp_df_1 = df_opt1.loc[df_opt1['vocab_size'] == '100word_list.txt'].copy()
temp_df_1.index= ["opt1"]
temp_df_1'''

In [ ]:
'''temp_df_2 = df_opt2.loc[df_opt2['vocab_size'] == '100word_list.txt'].copy()
temp_df_2.index= ["opt2"]
temp_df_2'''

In [ ]:
'''temp_df_3 = df_opt3.loc[df_opt3['vocab_size'] == '100word_list.txt'].copy()
temp_df_3.index= ["opt3"]
temp_df_3'''

In [ ]:
'''temp_df_4 = df_opt4.loc[df_opt4['vocab_size'] == '100word_list.txt'].copy()
temp_df_4.index= ["opt4"]
temp_df_4'''

In [ ]:
'''result = pd.concat([temp_df_1,temp_df_2,temp_df_3, temp_df_4])'''

In [ ]:
'''result'''

In [ ]:
'''result[['training_acc', 'validation_acc', 'testing_acc']].plot.bar(rot=0).legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
'''

In [ ]:
'''result[['training_acc', 'validation_acc', 'testing_acc']].plot.bar(rot=0).legend(loc='center left',bbox_to_anchor=(1.0, 0.5))
'''

In [43]:
for folder in results_folder_list[0:1]:
    file_path = folder + "/total_history_combined"
    with open(file_path) as json_file: 
        data = json.load(json_file) 
    with pd.ExcelWriter(folder+"/result.xlsx", mode='a') as writer:  
        for opt in ['opt1', 'opt2', 'opt3', 'opt4']:
            optdf = pd.DataFrame(data[opt])
            optdf.columns = ['Vocabulary Size', 'Training Accuracy', 'Validation Accuracy', 
                 'Testing Accuracy', 'Training Loss','Validation Loss' ,'Testing Loss']     
            print(optdf)#optdf.to_csv(folder+opt+'.csv')
            optdf.to_excel(writer, sheet_name=opt) 

ModuleNotFoundError: No module named 'openpyxl'

In [42]:
with open(file_path) as json_file: 
    data = json.load(json_file) 
print(file_path)
print(data['opt1'])
optdf = pd.DataFrame(data['opt1'])
optdf.columns = ['Vocabulary Size', 'Training Accuracy', 'Validation Accuracy', 
                 'Testing Accuracy', 'Training Loss','Validation Loss' ,'Testing Loss']            
print(optdf)
print(list(optdf))


/home/sukanya/PhD/Results/010_22_Feb_PAN_dataset_MPL_based/results/MPL_0/total_history_combined
{'vocab_size': ['50word_list.txt', '100word_list.txt', '150word_list.txt', '200word_list.txt', '250word_list.txt', '300word_list.txt', '350word_list.txt', '400word_list.txt', '450word_list.txt', '500word_list.txt'], 'training_acc': [0.5687500238418579, 0.59375, 0.5787500143051147, 0.5693749785423279, 0.5849999785423279, 0.5543749928474426, 0.5893750190734863, 0.5706250071525574, 0.5893750190734863, 0.5699999928474426], 'validation_acc': [0.5783227682113647, 0.5667194128036499, 0.5791139006614685, 0.5838607549667358, 0.5719936490058899, 0.5638185739517212, 0.565664529800415, 0.5896624326705933, 0.5741033554077148, 0.5825421810150146], 'testing_acc': [0.5863045454025269, 0.5766568779945374, 0.5923867225646973, 0.588716447353363, 0.5785444378852844, 0.5817952752113342, 0.5762374401092529, 0.6033976674079895, 0.5822147727012634, 0.5833683013916016], 'training_loss': [0.6754927635192871, 0.666117